In [80]:
from mediapipe.python.solutions import pose as mp_pose
import mediapipe as mp
import cv2
import os
import pandas as pd
import numpy as np

In [81]:
pose = mp_pose.Pose()
mpDraw = mp.solutions.drawing_utils

In [82]:
def distanceCal(heighPoint: float, heightMax: float) -> float:
    """Calculate the distance between each height and the minimum height.

    Args:
        heighPoint (float): Real y-coordinate of the keypoint
        heightMax (float): Maximum y-coordinate of the keypoint in first frame

    Returns:
        float: Distance from the keypoint to the ground
    """
    return abs(heighPoint - heightMax)

In [83]:
def findMax(heightList: np.array) -> int:
    """
    Find the minimum height in the height list.
    """
    return round(np.max(heightList), 10)

In [84]:
def getLandMark(result, poseName: str) -> float:
    """
    Return the current y-coordinate of the landmark.
    """
    if poseName.lower() == 'neck':
        return round(0.5* (getLandMark(result,'RIGHT_SHOULDER') + getLandMark(result,'LEFT_SHOULDER')), 10)
    elif poseName.lower() == 'torso':
        return round(0.5* (getLandMark(result, 'RIGHT_HIP') + getLandMark(result, 'LEFT_HIP')), 10) 
    return round(result.pose_world_landmarks.landmark[mp_pose.PoseLandmark._member_map_[poseName]].y,10)

In [105]:
def getFileKeypoints(path: str, choice: str) -> list:
    """
    This use to calculate max height of keypoints in the first frame and all y-coordinates of keypoints in the first frame.

    Returns:
        list: First list is the max height of keypoints in the first frame and the second list is all y-coordinates of keypoints in the first frame.
    """
    maxHeight = []
    data = []
    path = path + choice + '//'
    for i in range(1, len(os.listdir(path)) + 1):
        count = 0
        trainPath = path + 'video{}'.format(i) + '//'
        files = os.listdir(trainPath)
        keypoints = []
        img = cv2.imread(trainPath + '/' + files[count], cv2.IMREAD_COLOR)
        img = cv2.resize(img, (320,240))
        pose.reset()
        result = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        while not result.pose_landmarks:
            count += 1
            img = cv2.imread(trainPath + '/' + files[count], cv2.IMREAD_COLOR)
            img = cv2.resize(img, (320,240))
            pose.reset()
            result = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        for j in poseName:
            lm = getLandMark(result, j)
            keypoints.append(lm)
        maxHeight.append(findMax(keypoints[:len(keypoints) - 2]))
        data.append(keypoints)
    return maxHeight, data

In [86]:
def velocityCal(point1: float, point2: float, frame: int) -> float:
    """Calculate the velocity of the keypoint.

    Args:
        point1 (float): y-coordinate of the keypoint in the first frame.
        point2 (float): y-coordinate of the keypoint in every next frame.
        frame (int): frame to calculate the velocity.

    Returns:
        float: velocity of each keypoints in full video.
    """
    if frame == 1:
        return 0
    return round(((point2 - point1) / (float(frame) - 1)), 10)

In [87]:
def diffCal(point1: float, point2: float) -> float:
    """Calculate distance difference between each frame keypoints and first frame keypoints.

    Args:
        point1 (float): y-coordinate of the keypoint in the first frame.
        point2 (float): y-coordinate of the keypoint in every next frame.

    Returns:
        float: distance difference between each frame keypoints and first frame keypoints.
    """
    return round(point2 - point1, 10)

In [88]:
path = 'C://python//train//'

In [89]:
poseName = ['NOSE', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'neck', 'LEFT_ELBOW', 'RIGHT_ELBOW','LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_HIP', 'RIGHT_HIP','torso', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']

In [90]:
len(poseName)

17

In [106]:
maxHeightFall, keypointsFall = getFileKeypoints(path, 'fall')
maxHeightNor, keypointsNor = getFileKeypoints(path, 'normal')

In [92]:
def flatten(lst: list) -> list:
    return [x for xs in lst for x in xs]

In [93]:
def concatenate(a: list, b: list) -> list:
    c = list(map(list, zip(a, b)))
    return flatten(c)

## Velocity calculation approach

In [94]:
fallPath = path + 'fall//'
for i in range(1, len(os.listdir(fallPath)) + 1):
    frame = 1
    trainPath = fallPath + 'video{}'.format(i) + '//'
    files = os.listdir(trainPath)
    dataFall = []
    for file in files:
        dataTemp = []
        velocityData = []
        img = cv2.imread(trainPath + '/' + file, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (320,240))
        pose.reset()
        result = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if result.pose_landmarks:          
            for j in poseName[:len(poseName) - 2]:
                lm = getLandMark(result, j)
                velocity = velocityCal(distanceCal(lm, maxHeightFall[i-1]), distanceCal(keypointsFall[i-1][poseName.index(j)], maxHeightFall[i-1]), frame)
                dataTemp.append(distanceCal(lm, maxHeightFall[i-1]))
                velocityData.append(velocity)
            dataFall.append(concatenate(dataTemp, velocityData))
            frame += 1
        df = pd.DataFrame(dataFall)
        df.to_csv(path + 'fall_csv_new//video{}.csv'.format(i), index=False)

In [95]:
normalPath = path + 'normal//'
for i in range(1, len(os.listdir(normalPath)) + 1):
    frame = 1
    trainPath = normalPath + 'video{}'.format(i) + '//'
    files = os.listdir(trainPath)
    dataNor = []
    for file in files:
        dataTemp = []
        velocityData = []
        img = cv2.imread(trainPath + '/' + file, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (320,240))
        pose.reset()
        result = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if result.pose_landmarks:          
            for j in poseName[:len(poseName) - 2]:
                lm = getLandMark(result, j)
                velocity = velocityCal(distanceCal(lm, maxHeightNor[i-1]), distanceCal(keypointsNor[i-1][poseName.index(j)], maxHeightNor[i-1]), frame)
                dataTemp.append(distanceCal(lm, maxHeightNor[i-1]))
                velocityData.append(velocity)
            dataNor.append(concatenate(dataTemp, velocityData))
            frame += 1
        df = pd.DataFrame(dataNor)
        df.to_csv(path + 'normal_csv_new//video{}.csv'.format(i), index=False)

## Distance difference calculation approach

In [96]:
fallPath = path + 'fall//'
for i in range(1, len(os.listdir(fallPath)) + 1):
    trainPath = fallPath + 'video{}'.format(i) + '//'
    files = os.listdir(trainPath)
    dataFall = []
    for file in files:
        dataTemp = []
        differenceData = []
        img = cv2.imread(trainPath + '/' + file, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (320,240))
        pose.reset()
        result = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if result.pose_landmarks:          
            for j in poseName[:len(poseName) - 2]:
                lm = getLandMark(result, j)
                difference = diffCal(distanceCal(lm, maxHeightFall[i-1]), distanceCal(keypointsFall[i-1][poseName.index(j)], maxHeightFall[i-1]))
                dataTemp.append(distanceCal(lm, maxHeightFall[i-1]))
                differenceData.append(difference)
            dataFall.append(concatenate(dataTemp, differenceData))
        df = pd.DataFrame(dataFall)
        df.to_csv(path + 'fall_csv_new_test//video{}.csv'.format(i), index=False)

In [97]:
normalPath = path + 'normal//'
for i in range(1, len(os.listdir(normalPath)) + 1):
    frame = 1
    trainPath = normalPath + 'video{}'.format(i) + '//'
    files = os.listdir(trainPath)
    dataNor = []
    for file in files:
        dataTemp = []
        differenceData = []
        img = cv2.imread(trainPath + '/' + file, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (320,240))
        pose.reset()
        result = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if result.pose_landmarks:          
            for j in poseName[:len(poseName) - 2]:
                lm = getLandMark(result, j)
                difference = diffCal(distanceCal(lm, maxHeightNor[i-1]), distanceCal(keypointsNor[i-1][poseName.index(j)], maxHeightNor[i-1]))
                dataTemp.append(distanceCal(lm, maxHeightNor[i-1]))
                differenceData.append(difference)
            dataNor.append(concatenate(dataTemp, differenceData))
            frame += 1
        df = pd.DataFrame(dataNor)
        df.to_csv(path + 'normal_csv_new_test//video{}.csv'.format(i), index=False)